In [1]:
import sys
sys.path.append('..')

import os
import pandas as pd
import geopandas as gpd
from shapely import wkb
from datetime import datetime
from sqlalchemy import create_engine, text

from utilities_amigocloud import AmigocloudFunctions

from config import RUTA_UNIDAD_ONE_DRIVE
from config import RUTA_LOCAL_ONE_DRIVE
from config import API_AMIGOCLOUD_TOKEN_ADM
from config import POSTGRES_UTEA

RUTA_COMPLETA = os.path.join(RUTA_UNIDAD_ONE_DRIVE, RUTA_LOCAL_ONE_DRIVE)
ID_PROYECTO = 35248

In [2]:
amigocloud = AmigocloudFunctions(token=API_AMIGOCLOUD_TOKEN_ADM)
amigocloud

In [3]:
def obtener_engine():
    return create_engine(
        f"postgresql+psycopg2://{POSTGRES_UTEA['USER']}:{POSTGRES_UTEA['PASSWORD']}@{POSTGRES_UTEA['HOST']}:{POSTGRES_UTEA['PORT']}/{POSTGRES_UTEA['DATABASE']}"
    )

def obtener_parte_diario_amigocloud():
    query = '''SELECT parte.*, plan.unidad_01 
                FROM dataset_367119 parte 
                JOIN dataset_360912 plan 
                ON ST_Intersects(parte.referencia, plan.geometry);'''
    select = amigocloud.ejecutar_query_sql(ID_PROYECTO, query, 'get')
    return select['data']

def obtener_propiedad_plan_amigocloud(cod_prop):
    query = f'''SELECT * FROM dataset_360912 WHERE unidad_01 = {cod_prop}'''
    select = amigocloud.ejecutar_query_sql(ID_PROYECTO, query, 'get')
    return select['data']

def actualizar_lotes_db(unidad_01, unidad_03, ors, row):
    engine = obtener_engine()
    try:
        with engine.begin() as conn:
            query = text("""
                UPDATE drones_pulverizacion.parte_diario_pulv
                SET 
                    idd= :id,
                    fecha = :fecha,
                    semana = :semana,
                    temp = :temp,
                    viento = :viento,
                    humedad = :humedad,
                    cultivo = :tipo_cultivo,
                    desarrollo = :tipo_desarrollo,
                    estado = :estado,
                    id_generador = :id_generador,
                    id_vehiculo = :id_vehiculo
                WHERE unidad_01 = :unidad_01
                  AND unidad_03 = :unidad_03
                  AND os = :ors
            """)
            conn.execute(query, {
                "unidad_01": unidad_01,
                "unidad_03": unidad_03,
                "ors": ors,
                "id": row.id,
                "fecha": row.fecha_registro,
                "semana": datetime.fromisoformat(row.fecha_registro).isocalendar().week,
                "temp": row.temp,
                "viento": row.viento,
                "humedad": row.humedad,
                "tipo_cultivo": row.tipo_cultivo,
                "tipo_desarrollo": row.tipo_desarrollo,
                "estado" : 'EJECUTADO',
                "id_generador": row.id_generador,
                "id_vehiculo": row.id_vehiculo
                
            })
            print(f"✔️ Registros con unidad_01={unidad_01}, unidad_03={unidad_03}, os={ors} marcados como procesado.")
    except Exception as e:
        print(f"❌ Error al actualizar: {e}")

def copiar_parte_diario_a_clon_amigocloud(id):
    query = f"""INSERT INTO dataset_367584 (id, fecha_registro, temp, viento, humedad, observaciones, tipo_cultivo, tipo_desarrollo, referencia, id_generador, id_vehiculo) 
                SELECT id, fecha_registro, temp, viento, humedad, observaciones, tipo_cultivo, tipo_desarrollo, referencia, id_generador, id_vehiculo 
                FROM dataset_367119 
                WHERE id = {id}"""
    select = amigocloud.ejecutar_query_sql(ID_PROYECTO, query, 'post')
    return select

def eliminar_parte_diario_amigocloud(id):
    query = f"""DELETE FROM dataset_367119 WHERE id = {id}"""
    select = amigocloud.ejecutar_query_sql(ID_PROYECTO, query, 'post')
    return select

def copiar_planificacion_a_clon_amigocloud(unidad_01, ors):
    query = f"""INSERT INTO dataset_360915 (fecha_registro, id, unidad_01, unidad_02, unidad_03, unidad_04, unidad_05, area, os, geometry) 
                SELECT fecha_registro, id, unidad_01, unidad_02, unidad_03, unidad_04, unidad_05, area, os, geometry 
                FROM dataset_360912 
                WHERE unidad_01 = {unidad_01} and os = {ors}"""
    select = amigocloud.ejecutar_query_sql(ID_PROYECTO, query, 'post')
    return select

def eliminar_planificacion_amigocloud(unidad_01, ors):
    query = f"""DELETE FROM dataset_360912 WHERE unidad_01 = {unidad_01} and os = {ors}"""
    select = amigocloud.ejecutar_query_sql(ID_PROYECTO, query, 'post')
    return select

def convertir_json_a_geodataframe(json_data, nom_campo_geo):
    # Convertimos a DataFrame para manipulación tabular
    df = pd.DataFrame(json_data)
    # Convertimos la columna geometry (WKB hexadecimal) a objetos shapely
    df[nom_campo_geo] = df[nom_campo_geo].apply(lambda x: wkb.loads(bytes.fromhex(x)))
    # Creamos el GeoDataFrame en CRS original (EPSG:4326)
    gdf = gpd.GeoDataFrame(df, geometry=nom_campo_geo, crs='EPSG:4326')
    # Reproyectamos a UTM Zona 20S (EPSG:32720)
    gdf_utm20s = gdf.to_crs(epsg=32720)
    return gdf_utm20s

In [4]:
parte_diario = obtener_parte_diario_amigocloud()
parte_diario

[{'id': 101,
  'amigo_id': '8690a9e993e24703a2f1bb4186265911',
  'fecha_registro': '2026-02-07 21:19:59+00:00',
  'referencia': '0101000020E61000005F0A0F9A5D8B4FC0848C367D2C4931C0',
  'viento': 3.0,
  'humedad': 80.0,
  'observaciones': None,
  'tipo_cultivo': 'CAÑA DE AZUCAR',
  'tipo_desarrollo': 'CRECIMIENTO',
  'tipo': 'PARTE',
  'id_generador': 'GENERADOR DJI-01',
  'id_vehiculo': '6345TEY',
  'temp': 27.0,
  'unidad_01': 1},
 {'id': 98,
  'amigo_id': '8bb4dc2f5cd240eea64c573e3d8b08a1',
  'fecha_registro': '2026-02-07 21:15:16+00:00',
  'referencia': '0101000020E61000004589F1F510964FC0C0458204204931C0',
  'viento': 4.0,
  'humedad': 80.0,
  'observaciones': None,
  'tipo_cultivo': 'CAÑA DE AZUCAR',
  'tipo_desarrollo': 'CRECIMIENTO',
  'tipo': 'PARTE',
  'id_generador': 'GENERADOR TOYAMA-01',
  'id_vehiculo': '6375IEA',
  'temp': 27.0,
  'unidad_01': 19},
 {'id': 100,
  'amigo_id': 'f30be376eafd49aeb34b794160faff93',
  'fecha_registro': '2026-02-07 21:18:50+00:00',
  'referencia':

In [5]:
df_parte_diario = convertir_json_a_geodataframe(parte_diario, 'referencia')
df_parte_diario

,id,amigo_id,fecha_registro,referencia,viento,humedad,observaciones,tipo_cultivo,tipo_desarrollo,tipo,id_generador,id_vehiculo,temp,unidad_01
0,101,8690a9e993e24703a2f1bb4186265911,2026-02-07 21:19:59+00:00,POINT (490562.920 8088821.841),3.0,80.0,None,CAÑA DE AZUCAR,CRECIMIENTO,PARTE,GENERADOR DJI-01,6345TEY,27.0,1
1,98,8bb4dc2f5cd240eea64c573e3d8b08a1,2026-02-07 21:15:16+00:00,POINT (481677.976 8088836.876),4.0,80.0,None,CAÑA DE AZUCAR,CRECIMIENTO,PARTE,GENERADOR TOYAMA-01,6375IEA,27.0,19
2,100,f30be376eafd49aeb34b794160faff93,2026-02-07 21:18:50+00:00,POINT (485937.612 8092561.156),3.0,80.0,None,CAÑA DE AZUCAR,CRECIMIENTO,PARTE,GENERADOR TOYAMA-01,6375IEA,27.0,218
3,99,c89c8b536e0e41c2bf10ecaab5cccb5d,2026-02-07 21:17:21+00:00,POINT (472477.905 8090259.544),4.0,82.0,None,CAÑA DE AZUCAR,CRECIMIENTO,PARTE,GENERADOR DJI-01,6345TEY,27.0,217


In [6]:
for i, row in df_parte_diario.iterrows():
    cod_prop = row.unidad_01
    prop = obtener_propiedad_plan_amigocloud(cod_prop)
    gdf_prop = convertir_json_a_geodataframe(prop, 'geometry')
    
    # se queda con las combinacion unicas de unidad_01, unidad_03 y os
    gdf_prop_unicas = gdf_prop.drop_duplicates(subset=["unidad_01", "unidad_03", "os"])
    for j, row2 in gdf_prop_unicas.iterrows():
        print('os:', row2.os)
        actualizar_lotes_db(row2.unidad_01, row2.unidad_03, row2.os, row)
        copiar_planificacion_a_clon_amigocloud(row2.unidad_01, row2.os)
        eliminar_planificacion_amigocloud(row2.unidad_01, row2.os)
    copiar_parte_diario_a_clon_amigocloud(row.id)
    eliminar_parte_diario_amigocloud(row.id)
    print(f'SE PROCESO PROPIEDAD: {row2.unidad_02}')

os: 88
✔️ Registros con unidad_01=1, unidad_03=388, os=88 marcados como procesado.
SE PROCESO PROPIEDAD: SONIMA
os: 89
✔️ Registros con unidad_01=19, unidad_03=388, os=89 marcados como procesado.
SE PROCESO PROPIEDAD: SANTA BARBARA
os: 90
✔️ Registros con unidad_01=218, unidad_03=388, os=90 marcados como procesado.
SE PROCESO PROPIEDAD: DON QUIJOTE
os: 91
✔️ Registros con unidad_01=217, unidad_03=794, os=91 marcados como procesado.
SE PROCESO PROPIEDAD: FLAMBOYAN--AGUILERA
